In [1]:
import data_loader

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

import random
import normalizer
from PIL import Image, ImageDraw, ImageFont

import cv2

import models
import modelutils

In [2]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat, y))




train = data_loader.load("I:/NSU/CV/tests/torch/data/train/coords",
                         "I:/NSU/CV/tests/torch/data/train/images", 
                        firstn = 10000, batchSize = 32, shuffle = True, 
                        displace = True, size = 400, show = False)



scaler = normalizer.MinMaxNormalizer()
scaler.fit([y for _, y in train])

print("Number of batches:", len(train))
for x, y in train:
   print(x.shape, y.shape)
 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("devise is: ", device)



condcords12847_3d.txt dataimg12847.jpeg batch 1 from 312.5
400
0.78125 0.125 0.162109375
condcords15858_3d.txt dataimg15858.jpeg batch 1 from 312.5
400
0.78125 0.189453125 0.021484375
condcords23420_3d.txt dataimg23420.jpeg batch 1 from 312.5
400
0.78125 0.171875 0.041015625
condcords12405_3d.txt dataimg12405.jpeg batch 1 from 312.5
400
0.78125 0.11328125 0.0078125
condcords23089_3d.txt dataimg23089.jpeg batch 1 from 312.5
400
0.78125 0.17578125 0.095703125
condcords13789_3d.txt dataimg13789.jpeg batch 1 from 312.5
400
0.78125 0.111328125 0.158203125
condcords22132_3d.txt dataimg22132.jpeg batch 1 from 312.5
400
0.78125 0.173828125 0.1640625
condcords03518_3d.txt dataimg03518.jpeg batch 1 from 312.5
400
0.78125 0.173828125 0.0859375
condcords04844_3d.txt dataimg04844.jpeg batch 1 from 312.5
400
0.78125 0.212890625 0.19140625
condcords03468_3d.txt dataimg03468.jpeg batch 1 from 312.5
400
0.78125 0.146484375 0.0
condcords21419_3d.txt dataimg21419.jpeg batch 1 from 312.5
400
0.78125 0.160

In [3]:
train[0][0].shape

torch.Size([32, 3, 400, 400])

In [4]:
mouth_pointlist = [44, 7, 33, 14, 2, 31, 49, 15, 42, 32, 9, 51, 38, 61,
    18, 23, 12, 47, 67, 1, 2]
mouth_boundaries = [7, 14, 15, 67]
eye_L_pointlist = [62, 65, 0, 13, 34, 64]
eye_R_pointlist = [16, 36, 54, 55, 53, 63]
nose_pointlist = [5, 29, 51, 30, 4, 57]

eyes_corners = [53,36,62,13]

In [5]:

def coords_to_img(coordsbatch, idlist_list, imgsize, blobsize, bglevel):
    imglist = []
    for coords in coordsbatch:
        true_cord_list = []
        for idlist in idlist_list:
            midx = 0.0
            midy = 0.0
            for id in idlist:
                midx += coords[id][0]
                midy += coords[id][1]
            midx /= len(idlist)
            midy /= len(idlist)
            img = np.zeros((imgsize, imgsize), dtype=np.float32)
            x = int(midx * imgsize)
            y = int(midy * imgsize)
            true_cord_list.append(np.array([x, y]))

        for cord in true_cord_list:
            img[cord[1], cord[0]] = 15 * blobsize
        
        img = cv2.GaussianBlur(img, (351, 351), sigmaX=blobsize, borderType=cv2.BORDER_REPLICATE)
        
        for cord in true_cord_list:
            cv2.circle(img, (cord[0], cord[1]), 1, 6, -1)
        
    # img[y, x] = 15000
    # img = cv2.GaussianBlur(img, (351, 351), sigmaX=50, borderType=cv2.BORDER_REPLICATE)
    # cv2.circle(img, (x, y), 0, 6, -1)
    # img = cv2.GaussianBlur(img, (11, 11), sigmaX=1)

        img = cv2.GaussianBlur(img, (11, 11), sigmaX=2)
        img += bglevel
        imglist.append(torch.from_numpy(img).unsqueeze(0))
    return torch.stack(imglist)

In [6]:
def get_mean_coords(landmarks_list, tensor):
    x = torch.zeros(tensor.shape[0]).to(device)
    y = torch.zeros(tensor.shape[0]).to(device)
    for id in landmarks_list:
        x += tensor[:,id,0]
        y += tensor[:,id,1]
    x /= len(landmarks_list)
    y /= len(landmarks_list)
    return x, y

def lstolsofls(listoflist):
    newlist = []
    for listt in listoflist:
        newlist.append([listt])
    return newlist

In [7]:
class WMSELoss(nn.Module):
    def __init__(self,  device, l2_lambda = 0.001):
        super().__init__()
        self.device = device
        self.l2_lambda = l2_lambda
        
    def forward(self, x, y,weightmap, parameters = None):
        ls = ((x-y) * weightmap)**2
        sm = ls.sum() / (x.shape[0] * x.shape[1])
        if parameters is None:
            return sm
        
        pk = 0.0
        smp = 0.0
        if parameters is not None:
            for param in parameters:
                smp += (param**2).mean()
                pk += 1

        return sm + (smp / float(pk)) * self.l2_lambda

In [8]:
class CnnDetector(nn.Module):
    def __init__(self, device):
        super(CnnDetector, self).__init__()
        self.adpool = nn.AdaptiveAvgPool2d((64, 64)).to(device)
        self.pool = nn.MaxPool2d(2, 2).to(device)
        self.mid_depth = mid_depth = 7
        self.avpool = nn.AvgPool2d(2,2).to(device)
        self.conv1 = nn.Conv2d(3, mid_depth, 3, padding = 1).to(device) 
        self.conv2 = nn.Conv2d(mid_depth, mid_depth, 3, padding = 1).to(device)
        self.conv3 = nn.Conv2d(mid_depth, mid_depth, 3, padding = 1).to(device)
        self.conv4 = nn.Conv2d(mid_depth, mid_depth, 3, padding = 1).to(device)
        self.conv5 = nn.Conv2d(mid_depth, mid_depth, 3, padding = 1).to(device)
        self.conv6 = nn.Conv2d(mid_depth, mid_depth, 3, padding = 1).to(device)
        
        self.act = nn.ReLU().to(device)

    def forward(self, x):
        # Input: [batch, 3, H, W]
        x = self.adpool(x)
        x = self.pool(self.act(self.conv1(x)))
        x = self.pool(self.act(self.conv2(x)))
        x = self.pool(self.act(self.conv3(x)))
        x = self.pool(self.act(self.conv4(x)))
        x = self.pool(self.act(self.conv5(x)))
        x = self.avpool(self.conv6(x))
        x = F.Sigmoid()(x)
        return x


In [9]:
model = CnnDetector(device).to(device)

criterion = WMSELoss(device)

criterion = nn.MSELoss().to(device)

learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
def midcrop(x, y, border, imgtensor_batch):
    return imgtensor_batch[:, :, y-border:y+border, x-border:x+border]


def testmidctop():
    x = 100
    y = 100
    border = 20
    imgtensor_batch = train[0][0].to(device)
    
    cropped = midcrop(x, y, border, imgtensor_batch)
    print(cropped)
    cv2.imshow("cropped", cropped[0].cpu().numpy())
    cv2.waitKey(1000)

testmidctop()

tensor([[[[0.2745, 0.3059, 0.3020,  ..., 0.6941, 0.5843, 0.4784],
          [0.2510, 0.2745, 0.2824,  ..., 0.6706, 0.5569, 0.4627],
          [0.2314, 0.2667, 0.2784,  ..., 0.6431, 0.5294, 0.4510],
          ...,
          [0.3804, 0.3686, 0.3137,  ..., 0.1725, 0.2118, 0.2235],
          [0.2863, 0.2784, 0.2549,  ..., 0.1725, 0.2196, 0.2431],
          [0.2745, 0.2431, 0.2235,  ..., 0.1765, 0.2235, 0.2510]],

         [[0.2314, 0.2353, 0.2196,  ..., 0.5608, 0.4588, 0.3529],
          [0.2078, 0.2039, 0.2000,  ..., 0.5373, 0.4275, 0.3373],
          [0.1882, 0.1961, 0.1961,  ..., 0.5098, 0.4000, 0.3255],
          ...,
          [0.3255, 0.3137, 0.2588,  ..., 0.1059, 0.1216, 0.1176],
          [0.2392, 0.2314, 0.2078,  ..., 0.1059, 0.1294, 0.1373],
          [0.2275, 0.2078, 0.1882,  ..., 0.1020, 0.1333, 0.1451]],

         [[0.1608, 0.1490, 0.1137,  ..., 0.4549, 0.3765, 0.2941],
          [0.1373, 0.1176, 0.0941,  ..., 0.4314, 0.3529, 0.2863],
          [0.1176, 0.1098, 0.0902,  ..., 0

error: OpenCV(4.10.0) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0x85adec04::Set<3,4,-1>,struct cv::impl::A0x85adec04::Set<0,2,5>,4>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 40


: 

In [ ]:
learning_rate = 0.1

# learning loop
epoch_loss = 0
for epoch in range(20):
    epoch_loss = 0
    step_loss = 0
    step_count = 0
    random.shuffle(train)
    for batch_idx, (inputs, answers) in enumerate(train):
        needshow = torch.tensor(False).to(device)
        if(batch_idx % 3500 == 6):
           needshow = True

        eye_L_mid = get_mean_coords(eye_L_pointlist, answers)
        eye_R_mid = get_mean_coords(eye_R_pointlist, answers)
        
        inputs = inputs.to(device)
        answers = answers.to(device)

        fc = False

        outputs = model(inputs, fc)
        imganswers = coords_to_img(answers, [eye_L_pointlist, eye_R_pointlist], 64, 6, bglevel = 0).to(device)
        
        
        # weightmap = coords_to_img(answers, [eye_L_pointlist,eye_R_pointlist, mouth_pointlist], 64, 15, bglevel = 0.15).to(device)
        if(needshow):
            #xshow = inputs[0].cpu().numpy()
            yshow = (outputs[0][0].cpu().detach().numpy() * 255.0).astype('float32')
            # ansshow = (imganswers[0][0].cpu().detach().numpy() * 255.0).astype('float32')
            xshow = (inputs[0][0].cpu().detach().numpy() * 255.0).astype('float32')
            # weightshow = (weightmap[0][0].cpu().detach().numpy() * 255.0).astype('float32')
            # print(ansshow.max(), ansshow.min(), ansshow.mean(), ansshow.std())
            # print(type(ansshow[0][0]))
            yshow = (yshow - yshow.min()) * 255 / (yshow.max() - yshow.min())
            # ansshow = ansshow * 255 / ansshow.max()
            xshow = xshow * 255 / xshow.max()
            # weightshow = weightshow * 255 / weightshow.max()
            # pil_mod = Image.fromarray(yshow)
            # pil_mod.show()
            # print(yshow.max(), yshow.min(), yshow.mean(), yshow.std())
            # pil_imagey = Image.fromarray(ansshow)
            # pil_imagey.show()
            # pil_weighshow = Image.fromarray(weightshow)
            # pil_weighshow.show()
        
        loss = criterion(outputs, imganswers)
        step_loss += loss
        step_count += 1
        if(step_count == 10):
            optimizer.zero_grad()
            step_loss.backward()
            optimizer.step()
            print(f'Batch {batch_idx}, Loss: {step_loss.item() / step_count:.5f}')
            step_count = 0
            step_loss = 0

        epoch_loss += loss.item() 
        

        if batch_idx == 16:
            exit(0)
    
    #learning_rate /= 10.0
    #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print(f'shape {inputs.shape}, Epoch {epoch + 1}, Loss: {epoch_loss/len(train):.5f}')


In [ ]:

# learning loop
epoch_loss = 0
for epoch in range(20):
    epoch_loss = 0
    step_loss = 0
    step_count = 0
    random.shuffle(train)
    for batch_idx, (inputs, answers) in enumerate(train):
        needshow = torch.tensor(False).to(device)
        if(batch_idx % 3500 == 6):
           needshow = True

        eye_L_mid = get_mean_coords(eye_L_pointlist, answers)
        eye_R_mid = get_mean_coords(eye_R_pointlist, answers)
        
        inputs = inputs.to(device)
        answers = answers.to(device)

        fc = False

        outputs = model(inputs, fc)
        imganswers = coords_to_img(answers, [eye_L_pointlist, eye_R_pointlist], 64, 6, bglevel = 0).to(device)
        
        
        # weightmap = coords_to_img(answers, [eye_L_pointlist,eye_R_pointlist, mouth_pointlist], 64, 15, bglevel = 0.15).to(device)
        if(needshow):
            #xshow = inputs[0].cpu().numpy()
            yshow = (outputs[0][0].cpu().detach().numpy() * 255.0).astype('float32')
            # ansshow = (imganswers[0][0].cpu().detach().numpy() * 255.0).astype('float32')
            xshow = (inputs[0][0].cpu().detach().numpy() * 255.0).astype('float32')
            # weightshow = (weightmap[0][0].cpu().detach().numpy() * 255.0).astype('float32')
            # print(ansshow.max(), ansshow.min(), ansshow.mean(), ansshow.std())
            # print(type(ansshow[0][0]))
            yshow = (yshow - yshow.min()) * 255 / (yshow.max() - yshow.min())
            # ansshow = ansshow * 255 / ansshow.max()
            xshow = xshow * 255 / xshow.max()
            # weightshow = weightshow * 255 / weightshow.max()
            # pil_mod = Image.fromarray(yshow)
            # pil_mod.show()
            # print(yshow.max(), yshow.min(), yshow.mean(), yshow.std())
            # pil_imagey = Image.fromarray(ansshow)
            # pil_imagey.show()
            # pil_weighshow = Image.fromarray(weightshow)
            # pil_weighshow.show()
        
        loss = criterion(outputs, imganswers)
        step_loss += loss
        step_count += 1
        if(step_count == 10):
            optimizer.zero_grad()
            step_loss.backward()
            optimizer.step()
            print(f'Batch {batch_idx}, Loss: {step_loss.item() / step_count:.5f}')
            step_count = 0
            step_loss = 0

        epoch_loss += loss.item() 
        

        if batch_idx == 16:
            exit(0)
    
    #learning_rate /= 10.0
    #optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print(f'shape {inputs.shape}, Epoch {epoch + 1}, Loss: {epoch_loss/len(train):.5f}')


In [ ]:
# testloop
criterion = SelectiveRMSELoss(eye_L_pointlist + eye_R_pointlist, False, device)
epoch_loss = 0
test = data_loader.load("I:/NSU/CV/tests/torch/data/test/coords",
                        "I:/NSU/CV/tests/torch/data/test/images",  
                        firstn = 2000, batchSize = 16, shuffle = True, displace = True, 
                        size = 400, show = False)
random.shuffle(test)
                        


In [49]:
model

CnnDetector(
  (adpool): AdaptiveAvgPool2d(output_size=(128, 128))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv11): Conv2d(3, 7, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv21): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv31): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv41): Conv2d(7, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv12): Conv2d(3, 7, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv22): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv32): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv42): Conv2d(7, 1, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv13): Conv2d(3, 7, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv23): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv33): Conv2d(7, 7, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv43): 

In [ ]:
with torch.no_grad():
    for batch_idx, (inputs, answers) in enumerate(test):
        inputs = inputs.to(device)
        answers = answers.to(device)
        answers = scaler.transform(answers)
        outputs = model(inputs)
        outputs = scaler.inverse_transform(outputs)
        loss = criterion(outputs, answers)
        epoch_loss += loss.item()

    print(f'Test Loss: {epoch_loss/len(test):.4f}')


In [ ]:
random.shuffle(test)
with torch.no_grad():
    inputs, answers = test[1]
    inputs = inputs.to(device)
    answers = answers.to(device)
    outputs = model(inputs)
    outputs = scaler.inverse_transform(outputs)
    # x_l, y_l = get_mean_coords(eye_L_pointlist, outputs)
    # x_r, y_r = get_mean_coords(eye_R_pointlist, outputs)
    # print(x_l[0], y_l[0], x_r[0], y_r[0])
    # outputs[0][1][0] = x_l[0]
    # outputs[0][1][1] = y_l[0]
    # outputs[0][2][0] = x_r[0]
    # outputs[0][2][1] = y_r[0]
    print(outputs.shape, answers.shape)
    img = modelutils.show_tensor(inputs[0], outputs[0])
    img.show()
    imgdlib = modelutils.show_tensor(inputs[0], answers[0])
    imgdlib.show()



In [89]:


#Save the model after training
torch.save(model.state_dict(), "eye_area_detector_good.pth")


In [ ]:
class mouthBoundDetector(nn.Module):
    def __init__(self, device):
        super(mouthBoundDetector, self).__init__()
        self.last_detector_size = 128 
        self.adpool = nn.AdaptiveAvgPool2d((128, 128)).to(device) 
        self.conv1 = nn.Conv2d(3, 6, 3, padding = 1).to(device)
        self.pool = nn.MaxPool2d(2, 2).to(device)
        self.conv2 = nn.Conv2d(6, 9, 3, padding = 1).to(device)
        self.conv3 = nn.Conv2d(9, 20, 3, padding = 1).to(device)
        self.conv4 = nn.Conv2d(20, self.last_detector_size, 3, padding = 1).to(device)
        fcsize = 256
        self.fc1 = nn.Linear(self.last_detector_size*16*16, fcsize).to(device)
        self.fc_list = []
        for i in range(3):
            self.fc_list.append(nn.Linear(fcsize, fcsize).to(device))
        self.prelast = nn.Linear(fcsize, fcsize).to(device)
        self.fc_last = nn.Linear(fcsize, 3 * 68).to(device)
        self.act = nn.ReLU().to(device)
        self.sigm = nn.Sigmoid().to(device)
    
    def forward(self, x):
        # Input: [batch, 3, H, W]
        x = self.adpool(x)
        x = self.pool(self.act(self.conv1(x)))
        x = self.pool(self.act(self.conv2(x)))
        x = self.act(self.conv3(x))  
        x = self.pool(self.act(self.conv4(x)))
        x = x.view(-1, self.last_detector_size*16*16)       
        x = self.act(self.fc1(x))
        for i in range(len(self.fc_list)):
            x = self.act(self.fc_list[i](x))
        x = self.act(self.prelast(x))
        x = self.fc_last(x)
        x = x.view(-1, 68, 3)
        return x


In [ ]:
with torch.no_grad():
    inputs, answers = test[1]
    inputs = inputs.to(device)
    answers = answers.to(device)
    outputs = model(inputs)
    outputs = scaler.inverse_transform(outputs)
    print(outputs.shape, answers.shape)
    img = show_tensor(inputs[0], outputs[0])
    img.show()
    imgdlib = show_tensor(inputs[0], answers[0])
    imgdlib.show()